In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import time
import os
import pandas as pd

In [4]:
spark = SparkSession.builder \
    .appName("Test") \
    .config("spark.yarn.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
    .getOrCreate()

In [39]:
import cml.data_v1 as cmldata

#CONNECTION_NAME = "datalakesii-dev"
#conn = cmldata.get_connection(CONNECTION_NAME)#
#spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()


+------------------+
|         namespace|
+------------------+
|          apa_temp|
|           default|
|                dw|
|          dwbgdata|
|information_schema|
|            libsas|
|               sdj|
|               sys|
|               tmp|
+------------------+



In [36]:
EXAMPLE_SQL_QUERY = "select * from apa_temp.empdata"
spark.sql(EXAMPLE_SQL_QUERY).show()

+------+--------+-----------+
|emp_id|    name|emp_dept_id|
+------+--------+-----------+
|     3|Williams|         10|
|     4|   Jones|         30|
|     1|   Smith|         10|
|     2|    Rose|         20|
+------+--------+-----------+



In [7]:
empdataDF=spark.sql(EXAMPLE_SQL_QUERY)

In [8]:
type(empdataDF)

pyspark.sql.dataframe.DataFrame

In [9]:
empdataDF.show()

+------+--------+-----------+
|emp_id|    name|emp_dept_id|
+------+--------+-----------+
|     3|Williams|         10|
|     4|   Jones|         30|
|     1|   Smith|         10|
|     2|    Rose|         20|
+------+--------+-----------+



In [10]:
EXAMPLE_SQL_QUERY = "select * from  apa_temp.empdata e inner join apa_temp.deptdata d on ( e.emp_dept_id == d.dept_id)"
spark.sql(EXAMPLE_SQL_QUERY).show()

23/08/02 18:47:21 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     3|Williams|         10|  Finance|     10|
|     4|   Jones|         30|    Sales|     30|
|     1|   Smith|         10|  Finance|     10|
|     2|    Rose|         20|Marketing|     20|
+------+--------+-----------+---------+-------+



**lo rescato como DF y lo guardo**

In [11]:
cruceSimple=spark.sql(EXAMPLE_SQL_QUERY)

In [12]:
cruceSimple.show()

23/08/02 18:47:22 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     3|Williams|         10|  Finance|     10|
|     4|   Jones|         30|    Sales|     30|
|     1|   Smith|         10|  Finance|     10|
|     2|    Rose|         20|Marketing|     20|
+------+--------+-----------+---------+-------+



**acabo de hacer un cruce simple en spark, si deseo usar alguna librería clasica de ML podría querer pasar este DF de spark a pandas**

In [13]:
type(cruceSimple)

pyspark.sql.dataframe.DataFrame

In [14]:
cruceSimplePandas=cruceSimple.toPandas()

23/08/02 18:47:22 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.


In [15]:
type(cruceSimplePandas)

pandas.core.frame.DataFrame

In [16]:
cruceSimplePandas

,emp_id,name,emp_dept_id,dept_name,dept_id
0,3,Williams,10,Finance,10
1,4,Jones,30,Sales,30
2,1,Smith,10,Finance,10
3,2,Rose,20,Marketing,20


**podria querer guardar el archivo pandas como df spark**

In [17]:
#Create PySpark DataFrame from Pandas
cruceSimplePandas.iteritems = cruceSimplePandas.items
cruceSimplePandasSpark=spark.createDataFrame(cruceSimplePandas) 
cruceSimplePandasSpark.printSchema()
cruceSimplePandasSpark.show()

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- emp_dept_id: long (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)



+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     3|Williams|         10|  Finance|     10|
|     4|   Jones|         30|    Sales|     30|
|     1|   Smith|         10|  Finance|     10|
|     2|    Rose|         20|Marketing|     20|
+------+--------+-----------+---------+-------+



**ahora como spark lo puedo guardar otra vez**

In [18]:
project=os.environ["CDSW_PROJECT"]

In [19]:
rutaPersonal=project+'/tablas'
print(rutaPersonal)

prueba_hive_tablas/tablas


In [20]:
cruceSimplePandasSpark.write.mode('overwrite').option("path", "abfs://data@datalakesii.dfs.core.windows.net/DatosProcesados/%s/"%(rutaPersonal)+"cruceSimple").saveAsTable("apa_temp.cruceSimplePandasSpark")

23/08/02 18:47:26 WARN ShellBasedUnixGroupsMapping: unable to return groups for user manuel.barrientos
PartialGroupNameException The user name 'manuel.barrientos' is not found. id: ‘manuel.barrientos’: no such user
id: ‘manuel.barrientos’: no such user

	at org.apache.hadoop.security.ShellBasedUnixGroupsMapping.resolvePartialGroupNames(ShellBasedUnixGroupsMapping.java:291)
	at org.apache.hadoop.security.ShellBasedUnixGroupsMapping.getUnixGroups(ShellBasedUnixGroupsMapping.java:215)
	at org.apache.hadoop.security.ShellBasedUnixGroupsMapping.getGroupsSet(ShellBasedUnixGroupsMapping.java:123)
	at org.apache.hadoop.security.Groups$GroupCacheLoader.fetchGroupSet(Groups.java:413)
	at org.apache.hadoop.security.Groups$GroupCacheLoader.load(Groups.java:351)
	at org.apache.hadoop.security.Groups$GroupCacheLoader.load(Groups.java:300)
	at com.google.common.cache.LocalCache$LoadingValueReference.loadFuture(LocalCache.java:3529)
	at com.google.common.cache.LocalCache$Segment.loadSync(LocalCache.ja

**ahora podria querer leer la tabla que acabo de guardar**

In [21]:
EXAMPLE_SQL_QUERY = "select * from  apa_temp.cruceSimplePandasSpark"
spark.sql(EXAMPLE_SQL_QUERY).show()

+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     3|Williams|         10|  Finance|     10|
|     4|   Jones|         30|    Sales|     30|
|     1|   Smith|         10|  Finance|     10|
|     2|    Rose|         20|Marketing|     20|
+------+--------+-----------+---------+-------+

